In [14]:
import os
if "R_HOME" not in os.environ:
    os.environ['R_HOME'] = '/Library/Frameworks/R.framework/Resources/'
import numpy as np
import rpy2.robjects.numpy2ri
import rpy2.robjects as R
from rpy2.robjects.packages import importr
rpy2.robjects.numpy2ri.activate()
import pandas as pd
import statsmodels.stats.multitest as sm

R.r('set.seed')(1)

R_STATS = importr('stats')

def fisher_exact_2x2(matrix, alt):
    return R_STATS.fisher_test(matrix, alternative=alt)

In [5]:
gsm = pd.read_csv('../../data_tables/gsm/DLBCL.699.163drivers.Sep_23_2022.tsv', sep='\t', index_col=1)
gsm = gsm.drop('gene', axis=1)
gsm

,DLBCL11470,DLBCL10900,DLBC_FF_A7CQ_TP_NB,DLBCL10462,DLBCL_RICOVER_1081,DLBCL_LS1098,DLBCL_RICOVER_299,DLBCL11558,DLBCL_C_D_1105_NULLPAIR,DLBCL11447,...,DLBCL11455,DLBCL_RICOVER_685,DLBCL_LS146,DLBCL_RICOVER_111,DLBCL_RICOVER_173,DLBCL11515,DLBCL10491,DLBCL_RICOVER_1046,DLBCL10547,DLBCL10998
classifier_name,,,,,,,,,,,,,,,,,,,,,
SV.BCL6,3,0,0,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DTX1,2,1,0,0,0,0,0,0,2,0,...,1,0,0,0,0,0,0,0,0,0
CD70,2,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TNFAIP3,2,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2
NOTCH2,0,0,0,2,0,0,0,0,2,0,...,0,0,2,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X1Q32.1.AMP,0,0,0,0,1,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
BCL11A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
IRF4,0,2,2,0,0,0,0,0,0,0,...,2,0,0,1,0,0,0,0,0,0


In [11]:
labels = pd.read_csv('../../data_tables/confidence_tables/baseline_probabilities.connectivity_based.sensitivity_power2.Sep_23_2022.tsv',
                    sep='\t', index_col=0)
labels

,C1,C2,C3,C4,C5,cluster,confidence
DLBCL10877,0.985597,0.002210,0.000684,0.010885,0.000625,1,0.985597
DLBCL10928,0.984814,0.001006,0.000431,0.013195,0.000553,1,0.984814
DLBCL_MC_F648_JKW,0.984740,0.003482,0.000594,0.010454,0.000730,1,0.984740
DLBCL10844,0.984523,0.000563,0.000372,0.011383,0.003159,1,0.984523
DLBCL11501,0.983851,0.002258,0.000601,0.012017,0.001273,1,0.983851
...,...,...,...,...,...,...,...
DLBCL_C_D_PAIR20,0.000572,0.025097,0.384317,0.002925,0.587090,5,0.587090
DLBCL11504,0.159420,0.000438,0.000508,0.264150,0.575484,5,0.575484
DLBCL_RICOVER_150,0.000390,0.463520,0.000404,0.002014,0.533672,5,0.533672
DLBCL11538,0.003331,0.000994,0.228558,0.235071,0.532045,5,0.532045


16

In [39]:
tp53_biallelic = ((gsm.loc['TP53'] != 0) & (gsm.loc['X17P.DEL'] != 0)).astype(int)
myc_bcl2 = ((gsm.loc['SV.MYC'] != 0) & (gsm.loc['SV.BCL2'] != 0)).astype(int)

In [74]:
c2_bi = tp53_biallelic.loc[labels.loc[labels['cluster'] == 2].index].value_counts()
rest_bi = tp53_biallelic.loc[labels.loc[labels['cluster'] != 2].index].value_counts()

m = np.array([[0, 0]] * 2)
m[0, 0] = rest_bi[0]
m[1, 0] = rest_bi[1]
m[0, 1] = c2_bi[0]
m[1, 1] = c2_bi[1]

p_g = fisher_exact_2x2(m, 'two.sided')[0][0]

In [75]:
p_g, m

(7.987364899543694e-24,
 array([[500, 106],
        [ 28,  65]]))

In [86]:
c2_double = myc_bcl2.loc[labels.loc[labels['cluster'] == 3].index].value_counts()
rest_double = myc_bcl2.loc[labels.loc[labels['cluster'] != 3].index].value_counts()

m = np.array([[0, 0]] * 2)
m[0, 0] = c2_double[0]
m[1, 0] = c2_double[1]
m[0, 1] = rest_double[0]
m[1, 1] = rest_double[1]

p_g = fisher_exact_2x2(m, 'two.sided')[0][0]
p_g, m

(1.610426348613921e-08,
 array([[104, 579],
        [ 13,   3]]))

In [85]:
labels.loc[gsm.columns[((gsm.loc['SV.MYC'] == 3) & (gsm.loc['SV.BCL2'] == 3))], 'cluster'].value_counts()

3    13
4     2
2     1
Name: cluster, dtype: int64